In [1]:
from translation_machine import dataset_mod,sentence_mod

import numpy as np
import torch

language_info = torch.load("../models/language_info.pth")

vocab_french = language_info["french"]["vocab"]
vocab_english = language_info["english"]["vocab"]

max_length_french = language_info["french"]["max_sentence_train_val"]
max_length_english = language_info["english"]["max_sentence_train_val"]


whole_dataset = dataset_mod.DatasetFromTxt("../data/french_english_dataset/fra.txt")

whole_dataset = list(dataset_mod.SentenceDataSet(whole_dataset,sentence_type_src=sentence_mod.EnglishSentence,sentence_type_dst=sentence_mod.FrenchSentence))




In [2]:
from translation_machine import collate_fn_mod

import torch
from torch.utils.data import DataLoader
import numpy as np

collate_fn = collate_fn_mod.get_collate_fn(max_length_english,max_length_french)

batch_size= 16

data_loader = DataLoader(whole_dataset,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)


In [3]:
from translation_machine.models import transformer_mod
model = transformer_mod.TransformerForSeq2Seq(64,sentence_mod.EnglishSentence.vocab,sentence_mod.FrenchSentence.vocab)


In [4]:
for batch in data_loader:
    break

In [5]:
batch

[tensor([[   6,  158,  353,  ...,    0,    0,    0],
         [   6,   89,  432,  ...,    0,    0,    0],
         [ 133,  752,  966,  ...,    0,    0,    0],
         ...,
         [2602,   29, 3506,  ...,    0,    0,    0],
         [  78,   79,  354,  ...,    0,    0,    0],
         [1472,  241,    0,  ...,    0,    0,    0]]),
 tensor([[   1,    9,  325,  ...,    0,    0,    0],
         [   1,    7,    8,  ...,    0,    0,    0],
         [   1,  187, 1379,  ...,    0,    0,    0],
         ...,
         [   1,    0,   34,  ...,    0,    0,    0],
         [   1,   91, 2411,  ...,    0,    0,    0],
         [   1,  380,  150,  ...,    0,    0,    0]]),
 tensor([ 9,  6, 15,  5,  9,  5,  5,  5,  4,  8,  5,  6,  5, 14,  9,  4]),
 tensor([13,  8, 14,  7, 11,  8,  8,  7,  6, 10,  8,  8,  8, 17,  9,  7])]

In [6]:
model = model.to("cuda")
batch[:2] = [el.to("cuda") for el in batch[:2]]

In [7]:
res = model.forward(batch)

In [9]:
assert  all(el in res.keys() for el in ['main', 'target_model'])

In [21]:
assert (res["main"].shape[0] == res["target_model"].shape[0])

In [69]:
#directly from batch
target_model = torch.concat([instance[1:length] for instance,length in zip(batch[1],batch[3])])

In [70]:
assert torch.all(target_model == res["target_model"])